###**필요한 package 불러오기**

In [ ]:
import io
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.metrics import f1_score
import keras
from tensorflow.keras.utils import to_categorical 
#from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

###**Original Data trainset에서 2015년~2016년 데이터만 가져오기 - validation set**

In [ ]:
# Original trainset 가져오기
from google.colab import files
myfiles=files.upload()

# trainset.csv 파일 불러오기

Saving trainset.csv to trainset.csv


In [ ]:
data_valid = pd.read_csv(io.BytesIO(myfiles['trainset.csv']))
data_valid.head()

,Id,C,Y_c,Y_b,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,...,BX24,BX25,BX26,BX27,BX28,BX29,BX30,BX31,BX32,BX33,BX34,BX35,BX36,BX37,BX38,BX39,BX40,BX41,BX42,BX43,BX44,BX45,BX46,BX47,BX49,BX50,BX51,BX52,BX53,BX54,BX55,BX56,BX57,BX58,BX59,BX60,yy_c,mm_c,yy_b,mm_b
0,a1,-1,BBB,AA+,22.383724,22.128057,20.546765,22.210202,8.331105,1.041042,-0.382176,58,0.174074,13.796952,-7.150123,-0.024824,0.010184,0.063153,0.039737,0.047798,0.018831,-0.621855,0.065031,0.064817,1.053593,-0.003308,-0.003841,0.114455,0.061723,-0.004960,0.885545,0.052732,-0.016081,17.013179,0.159301,0.010184,0.267607,0.234077,1.530844,0.713886,...,0.115882,0.056666,-0.004603,0.884118,0.059216,-0.065824,14.162630,0.196878,0.016816,0.272419,0.241213,1.622491,0.753791,0.506855,0.993417,1.007513,3.869614,2.707154,1.372134,0.330942,0.632885,0.245141,-0.003509,0.024012,18.715408,0.780712,1.153126,0.010272,0.008250,-0.065530,-0.072887,15.046856,1.609134,0.722055,2.950170,7.179745,1998,6,1997,6
1,a2,-1,B,A,20.184873,19.906590,18.216023,20.034495,6.315358,0.880409,-0.545045,43,0.174074,13.591232,5.784040,0.010394,0.017520,0.057560,0.048016,0.042984,0.018831,0.189408,0.081739,0.079748,1.439674,-0.024963,0.001451,0.100284,0.056776,0.001917,0.899716,0.043508,-0.135363,17.013179,0.139617,0.017520,0.076551,0.136176,1.596068,0.945048,...,0.104049,0.062658,0.002779,0.895951,0.041390,-0.082338,12.543782,0.127709,0.019913,0.076551,0.258583,0.929468,0.933123,0.654066,1.258566,0.711496,6.123913,4.034517,2.309551,0.297859,0.708990,0.146406,0.136358,0.007096,18.223464,0.844509,0.563029,-0.030763,-0.026834,-0.034293,-0.036829,13.671127,1.444607,0.812241,2.966269,4.625441,1998,6,1997,6
2,a3,0,BBB,BBB,21.026399,19.623518,17.221982,21.170458,7.671361,0.979021,-0.545045,42,0.174074,12.172528,-0.267213,0.240345,-0.129443,-0.041703,0.085964,-0.034289,0.003062,-5.264545,0.104237,0.097745,-0.945759,-0.116040,-0.101932,0.038197,-0.047666,-0.146520,0.961803,0.113158,0.274954,17.013179,0.118339,-0.129443,0.692466,0.011330,1.091126,0.319425,...,0.087675,0.012465,-0.146520,0.912325,0.075210,-0.333624,12.822377,0.118339,-0.070151,0.692466,0.342592,0.689131,0.346003,1.229036,0.526314,1.361668,6.123913,4.034517,2.309551,0.344134,0.751389,0.133747,-0.205083,-0.023834,17.572121,0.058201,0.435633,-0.039685,-0.038299,-0.181887,-0.156031,37.349959,1.013982,0.370970,2.727980,5.430017,1998,6,1997,6
3,a4,-1,BBB-,A+,21.322081,21.067553,20.126214,20.961911,8.131825,1.137434,-0.075908,42,0.174074,12.935729,-5.193739,0.004219,0.022513,0.074403,0.052825,0.057377,0.018831,0.138225,0.086533,0.085463,1.169247,-0.012526,0.001276,0.111693,0.074007,0.001646,0.888307,0.037686,-0.032108,14.673233,0.302442,0.022513,0.451714,0.198101,1.650138,0.598405,...,0.106502,0.069972,0.007435,0.893498,0.036530,-0.104062,6.313040,0.132332,0.030286,0.354025,0.241267,1.717064,0.653632,0.469319,0.861478,1.199768,5.899879,4.034517,2.094303,0.382395,0.660291,0.152515,-0.081800,0.029256,19.164520,0.830141,2.110466,0.101802,0.088331,0.070543,0.065115,9.999484,1.923207,0.939994,4.854726,5.030613,1998,6,1997,6
4,a5,0,AA-,AA-,19.975171,19.635487,18.909031,19.553065,6.858565,0.932399,0.008580,42,0.239319,12.776922,-33.644871,-0.043416,0.004551,0.046257,0.049430,0.027739,0.011753,-3.537565,0.065937,0.064205,1.692427,-0.026977,-0.014950,0.188970,0.038960,-0.020997,0.811030,0.150010,-0.055782,7.956532,0.344335,0.004551,0.357530,0.335760,1.334028,0.875906,...,0.166282,0.035335,-0.037372,0.833718,0.130947,-0.063286,4.588228,0.339781,0.027357,0.497970,0.381397,2.939148,0.557548,0.334706,0.901301,1.096611,1.762613,1.455807,0.487268,0.273604,0.411969,0.514648,-0.048822,0.004076,18.744605,0.600945,3.007393,0.134023,0.088484,0.069055,0.071358,15.552325,1.141874,0.638910,3.570284,8.855187,1998,6,1997,6


In [ ]:
# 불필요한 열 삭제
data_valid=data_valid.drop(columns=['Id','Y_c','yy_c','mm_c','mm_b'])
data_valid.head()

,C,Y_b,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,...,BX21,BX22,BX23,BX24,BX25,BX26,BX27,BX28,BX29,BX30,BX31,BX32,BX33,BX34,BX35,BX36,BX37,BX38,BX39,BX40,BX41,BX42,BX43,BX44,BX45,BX46,BX47,BX49,BX50,BX51,BX52,BX53,BX54,BX55,BX56,BX57,BX58,BX59,BX60,yy_b
0,-1,AA+,22.383724,22.128057,20.546765,22.210202,8.331105,1.041042,-0.382176,58,0.174074,13.796952,-7.150123,-0.024824,0.010184,0.063153,0.039737,0.047798,0.018831,-0.621855,0.065031,0.064817,1.053593,-0.003308,-0.003841,0.114455,0.061723,-0.004960,0.885545,0.052732,-0.016081,17.013179,0.159301,0.010184,0.267607,0.234077,1.530844,0.713886,0.493392,0.932059,...,1.316730,-0.017948,-0.003323,0.115882,0.056666,-0.004603,0.884118,0.059216,-0.065824,14.162630,0.196878,0.016816,0.272419,0.241213,1.622491,0.753791,0.506855,0.993417,1.007513,3.869614,2.707154,1.372134,0.330942,0.632885,0.245141,-0.003509,0.024012,18.715408,0.780712,1.153126,0.010272,0.008250,-0.065530,-0.072887,15.046856,1.609134,0.722055,2.950170,7.179745,1997
1,-1,A,20.184873,19.906590,18.216023,20.034495,6.315358,0.880409,-0.545045,43,0.174074,13.591232,5.784040,0.010394,0.017520,0.057560,0.048016,0.042984,0.018831,0.189408,0.081739,0.079748,1.439674,-0.024963,0.001451,0.100284,0.056776,0.001917,0.899716,0.043508,-0.135363,17.013179,0.139617,0.017520,0.076551,0.136176,1.596068,0.945048,0.439255,1.094028,...,1.206611,-0.012946,0.002257,0.104049,0.062658,0.002779,0.895951,0.041390,-0.082338,12.543782,0.127709,0.019913,0.076551,0.258583,0.929468,0.933123,0.654066,1.258566,0.711496,6.123913,4.034517,2.309551,0.297859,0.708990,0.146406,0.136358,0.007096,18.223464,0.844509,0.563029,-0.030763,-0.026834,-0.034293,-0.036829,13.671127,1.444607,0.812241,2.966269,4.625441,1997
2,0,BBB,21.026399,19.623518,17.221982,21.170458,7.671361,0.979021,-0.545045,42,0.174074,12.172528,-0.267213,0.240345,-0.129443,-0.041703,0.085964,-0.034289,0.003062,-5.264545,0.104237,0.097745,-0.945759,-0.116040,-0.101932,0.038197,-0.047666,-0.146520,0.961803,0.113158,0.274954,17.013179,0.118339,-0.129443,0.692466,0.011330,1.091126,0.319425,0.776332,0.495617,...,2.397373,-0.116040,-0.065861,0.087675,0.012465,-0.146520,0.912325,0.075210,-0.333624,12.822377,0.118339,-0.070151,0.692466,0.342592,0.689131,0.346003,1.229036,0.526314,1.361668,6.123913,4.034517,2.309551,0.344134,0.751389,0.133747,-0.205083,-0.023834,17.572121,0.058201,0.435633,-0.039685,-0.038299,-0.181887,-0.156031,37.349959,1.013982,0.370970,2.727980,5.430017,1997
3,-1,A+,21.322081,21.067553,20.126214,20.961911,8.131825,1.137434,-0.075908,42,0.174074,12.935729,-5.193739,0.004219,0.022513,0.074403,0.052825,0.057377,0.018831,0.138225,0.086533,0.085463,1.169247,-0.012526,0.001276,0.111693,0.074007,0.001646,0.888307,0.037686,-0.032108,14.673233,0.302442,0.022513,0.451714,0.198101,1.650138,0.598405,0.451540,0.746235,...,1.209369,-0.014650,0.006989,0.106502,0.069972,0.007435,0.893498,0.036530,-0.104062,6.313040,0.132332,0.030286,0.354025,0.241267,1.717064,0.653632,0.469319,0.861478,1.199768,5.899879,4.034517,2.094303,0.382395,0.660291,0.152515,-0.081800,0.029256,19.164520,0.830141,2.110466,0.101802,0.088331,0.070543,0.065115,9.999484,1.923207,0.939994,4.854726,5.030613,1997
4,0,AA-,19.975171,19.635487,18.909031,19.553065,6.858565,0.932399,0.008580,42,0.239319,12.776922,-33.644871,-0.043416,0.004551,0.046257,0.049430,0.027739,0.011753,-3.537565,0.065937,0.064205,1.692427,-0.026977,-0.014950,0.188970,0.038960,-0.020997,0.811030,0.150010,-0.055782,7.956532,0.344335,0.004551,0.357530,0.335760,1.334028,0.875906,0.635180,1.318658,...,1.952505,-0.033657,-0.023878,0.166282,0.035335,-0.037372,0.833718,0.130947,-0.063286,4.588228,0.339781,0.027357,0.497970,0.381397,2.939148,0.557548,0.334706,0.901301,1.096611,1.762613,1.455807,0.487268,0.273604,0.411969,0.514648,-0.048822,0.004076,18.744605,0.600945,3.007393,0.134023,0.088484,0.069055,0.071358,15.552325,1.141874,0.638910,3.570284,8.855187,1997


In [ ]:
# validation set 가져오기
valid=data_valid[data_valid['yy_b']>2013] # 398개
valid=pd.DataFrame(valid)
valid['yy_b'].value_counts()

2014    205
2015    193
Name: yy_b, dtype: int64

In [ ]:
# Y_b를 one-hot encoding
valid['Y_b']=valid['Y_b'].replace({'D':'0', 'C':'1','CC':'2','CCC-':'3','CCC':'4',
                                 'B-':'5','B':'6','B+':'7','BB-':'8','BB':'9',
                                 'BB+':'10','BBB-':'11','BBB':'12','BBB+':'13',
                                 'A-':'14','A':'15','A+':'16','AA-':'17','AA':'18',
                                 'AA+':'19','AAA':'20'})

valid['Y_b'] = valid['Y_b'].astype('int64')


In [ ]:
valid.head()

,C,Y_b,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,...,BX21,BX22,BX23,BX24,BX25,BX26,BX27,BX28,BX29,BX30,BX31,BX32,BX33,BX34,BX35,BX36,BX37,BX38,BX39,BX40,BX41,BX42,BX43,BX44,BX45,BX46,BX47,BX49,BX50,BX51,BX52,BX53,BX54,BX55,BX56,BX57,BX58,BX59,BX60,yy_b
2547,0,17,21.751434,21.517781,21.226698,20.855654,5.036953,0.550943,0.245995,65,0.472310,16.480828,3.208753,0.012707,0.305364,0.034193,0.000788,0.022812,0.004888,2.452472,0.006963,0.007118,0.241621,0.021854,0.007512,0.157718,0.028817,0.009489,0.842282,0.128902,0.029238,1.462278,0.591711,0.305364,0.345000,0.250155,1.262619,1.242270,0.685338,1.656703,...,2.397373,-0.006313,-0.010099,0.130174,0.001061,-0.011252,0.869826,0.129112,-0.008867,1.511415,0.638985,0.335644,0.368482,0.263084,2.134732,1.183863,0.508644,1.606508,0.809208,0.473711,0.374496,0.190485,0.134829,0.218308,1.769968,0.145136,0.009746,18.087333,0.836400,-0.351821,-0.024784,-0.008947,-0.100653,-0.040486,37.349959,2.043695,0.897528,11.528939,13.442271,2014
2548,-1,16,21.953598,21.350451,21.006886,21.462550,8.108623,0.705690,0.086414,60,1.248586,13.241828,-1.960047,0.016074,0.100362,0.062451,0.028478,0.027380,0.000458,0.303047,0.029133,0.029755,0.582113,0.020914,0.006210,0.418655,0.050046,0.011351,0.581345,0.368609,0.029488,1.268626,0.388015,0.100362,0.667135,0.185009,1.132678,0.455584,0.598178,0.559005,...,0.350435,0.055133,0.022263,0.433632,0.084877,0.041689,0.566368,0.348755,0.075636,1.386416,0.389266,0.118920,0.658012,0.177105,1.367152,0.469710,0.553376,0.570801,1.278076,1.523229,1.010016,0.558918,0.213158,0.438572,0.637375,-0.168746,0.056347,19.647054,3.235508,5.358990,0.113369,0.069238,0.070734,0.080894,8.533729,2.029431,0.534025,3.565043,7.776367,2014
2549,0,17,22.236822,22.240608,21.547731,21.539602,8.575651,0.862353,0.917266,65,1.960493,13.664957,9.354185,0.026068,0.096706,0.043015,0.011048,0.036779,0.003374,2.606121,0.014409,0.014737,0.393276,0.022230,0.013087,0.104954,0.036640,0.013037,0.895046,0.068314,0.044448,0.505946,0.502032,0.096706,0.394723,0.011330,0.226798,1.442380,1.600284,1.453490,...,0.933007,0.001133,-0.013032,0.078404,0.016911,-0.015826,0.921596,0.061493,0.001912,1.034753,0.487816,0.086628,0.394136,0.011330,0.281269,1.373918,1.466592,1.389440,0.897941,1.004855,0.425670,0.624284,0.079949,0.388913,0.952423,0.080867,0.005139,16.862080,1.338676,-0.232977,-0.031276,-0.016019,-0.102062,-0.063482,22.360964,5.069560,0.823449,5.471615,10.236531,2014
2550,-1,15,22.114047,21.359708,20.813324,21.796133,4.729156,0.788839,0.094757,65,0.263140,16.507898,-1.590654,-0.092600,0.145802,0.067222,0.045361,0.026899,0.000458,-4.356859,0.046181,0.046695,0.807454,0.011012,-0.025164,0.421590,0.057193,-0.053504,0.578410,0.364397,0.019018,10.176007,0.272335,0.145802,0.568013,0.182802,1.255040,0.342780,0.619390,0.495617,...,0.593418,0.035137,-0.012148,0.431145,0.086421,-0.026650,0.568855,0.344724,0.055129,11.071560,0.290536,0.170258,0.547937,0.171245,1.884681,0.338043,0.474745,0.495617,1.545983,2.383703,1.655822,0.786094,0.329586,0.635277,0.409514,-0.239250,0.014446,19.607197,1.869619,3.335384,0.076953,0.054595,0.046417,0.072243,14.534991,2.043407,0.455843,3.618095,8.061703,2014
2551,-1,16,24.080389,23.734495,22.909580,23.603021,8.278936,1.139624,-0.232024,65,0.284130,15.455559,34.072022,0.003746,0.057898,0.055944,0.031573,0.031813,0.002047,1.563144,0.034734,0.035241,0.706991,0.014395,0.001060,0.166081,0.049130,0.001637,0.833919,0.116952,0.032846,8.909248,0.283792,0.057898,0.273628,0.232075,0.785788,0.745383,0.776287,0.970645,...,0.682763,0.016660,0.004177,0.161956,0.052516,0.006022,0.838044,0.109440,0.039757,6.876695,0.290636,0.059454,0.277866,0.210541,0.607905,0.822465,0.871131,1.041808,0.974472,2.301794,1.304412,1.136314,0.159201,0.475329,0.409714,0.015850,0.024279,19.985608,1.620947,1.494623,0.017340,0.012301,-0.000152,-0.000156,11.791604,1.944470,0.693577,7.148437,7.458647,2014


###**균형 데이터 불러오기 - smote_data_perfect2**

In [ ]:
from google.colab import files
myfile=files.upload()

#perfect_data.csv 파일 불러오기

Saving smote_data_perfect2.csv to smote_data_perfect2.csv


In [ ]:
perfect_data = pd.read_csv(io.BytesIO(myfile['perfect_data.csv']))
perfect_data=perfect_data.drop(columns=['Unnamed: 0'])
perfect_data.head()

,Y_b,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,BX22,BX23,BX24,BX25,BX26,BX27,BX28,BX29,BX30,BX31,BX32,BX33,BX34,BX35,BX36,BX37,BX38,BX39,BX40,BX41,BX42,BX43,BX44,BX45,BX46,BX47,BX49,BX50,BX51,BX52,BX53,BX54,BX55,BX56,BX57,BX58,BX59,BX60,yy_b,C
0,19,22.383724,22.128057,20.546765,22.210202,8.331105,1.041042,-0.382176,58,0.174074,13.796952,-7.150123,-0.024824,0.010184,0.063153,0.039737,0.047798,0.018831,-0.621855,0.065031,0.064817,1.053593,-0.003308,-0.003841,0.114455,0.061723,-0.004960,0.885545,0.052732,-0.016081,17.013179,0.159301,0.010184,0.267607,0.234077,1.530844,0.713886,0.493392,0.932059,1.088378,...,-0.017948,-0.003323,0.115882,0.056666,-0.004603,0.884118,0.059216,-0.065824,14.162630,0.196878,0.016816,0.272419,0.241213,1.622491,0.753791,0.506855,0.993417,1.007513,3.869614,2.707154,1.372134,0.330942,0.632885,0.245141,-0.003509,0.024012,18.715408,0.780712,1.153126,0.010272,0.008250,-0.065530,-0.072887,15.046856,1.609134,0.722055,2.950170,7.179745,1997,-1
1,15,20.184873,19.906590,18.216023,20.034495,6.315358,0.880409,-0.545045,43,0.174074,13.591232,5.784040,0.010394,0.017520,0.057560,0.048016,0.042984,0.018831,0.189408,0.081739,0.079748,1.439674,-0.024963,0.001451,0.100284,0.056776,0.001917,0.899716,0.043508,-0.135363,17.013179,0.139617,0.017520,0.076551,0.136176,1.596068,0.945048,0.439255,1.094028,0.860245,...,-0.012946,0.002257,0.104049,0.062658,0.002779,0.895951,0.041390,-0.082338,12.543782,0.127709,0.019913,0.076551,0.258583,0.929468,0.933123,0.654066,1.258566,0.711496,6.123913,4.034517,2.309551,0.297859,0.708990,0.146406,0.136358,0.007096,18.223464,0.844509,0.563029,-0.030763,-0.026834,-0.034293,-0.036829,13.671127,1.444607,0.812241,2.966269,4.625441,1997,-1
2,12,21.026399,19.623518,17.221982,21.170458,7.671361,0.979021,-0.545045,42,0.174074,12.172528,-0.267213,0.240345,-0.129443,-0.041703,0.085964,-0.034289,0.003062,-5.264545,0.104237,0.097745,-0.945759,-0.116040,-0.101932,0.038197,-0.047666,-0.146520,0.961803,0.113158,0.274954,17.013179,0.118339,-0.129443,0.692466,0.011330,1.091126,0.319425,0.776332,0.495617,1.545983,...,-0.116040,-0.065861,0.087675,0.012465,-0.146520,0.912325,0.075210,-0.333624,12.822377,0.118339,-0.070151,0.692466,0.342592,0.689131,0.346003,1.229036,0.526314,1.361668,6.123913,4.034517,2.309551,0.344134,0.751389,0.133747,-0.205083,-0.023834,17.572121,0.058201,0.435633,-0.039685,-0.038299,-0.181887,-0.156031,37.349959,1.013982,0.370970,2.727980,5.430017,1997,0
3,16,21.322081,21.067553,20.126214,20.961911,8.131825,1.137434,-0.075908,42,0.174074,12.935729,-5.193739,0.004219,0.022513,0.074403,0.052825,0.057377,0.018831,0.138225,0.086533,0.085463,1.169247,-0.012526,0.001276,0.111693,0.074007,0.001646,0.888307,0.037686,-0.032108,14.673233,0.302442,0.022513,0.451714,0.198101,1.650138,0.598405,0.451540,0.746235,1.256871,...,-0.014650,0.006989,0.106502,0.069972,0.007435,0.893498,0.036530,-0.104062,6.313040,0.132332,0.030286,0.354025,0.241267,1.717064,0.653632,0.469319,0.861478,1.199768,5.899879,4.034517,2.094303,0.382395,0.660291,0.152515,-0.081800,0.029256,19.164520,0.830141,2.110466,0.101802,0.088331,0.070543,0.065115,9.999484,1.923207,0.939994,4.854726,5.030613,1997,-1
4,17,19.975171,19.635487,18.909031,19.553065,6.858565,0.932399,0.008580,42,0.239319,12.776922,-33.644871,-0.043416,0.004551,0.046257,0.049430,0.027739,0.011753,-3.537565,0.065937,0.064205,1.692427,-0.026977,-0.014950,0.188970,0.038960,-0.020997,0.811030,0.150010,-0.055782,7.956532,0.344335,0.004551,0.357530,0.335760,1.334028,0.875906,0.635180,1.318658,0.786697,...,-0.033657,-0.023878,0.166282,0.035335,-0.037372,0.833718,0.130947,-0.063286,4.588228,0.339781,0.027357,0.497970,0.381397,2.939148,0.557548,0.334706,0.901301,1.096611,1.762613,1.455807,0.487268,0.273604,0.411969,0.514648,-0.048822,0.004076,18.744605,0.600945,3.007393,0.134023,0.088484,0.069055,0.071358,15.552325,1.141874,0.638910,3.570284,8.855187,1997,0


###**Random Forest 적용해보기**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# train set과 validation set 지정
x_train=perfect_data.drop(columns=['C']).astype(float)
y_train = perfect_data['C'].astype('category')
  
x_test=valid.drop(columns=['C']).astype(float)
y_test=valid['C'].astype('category')

In [ ]:
print(y_train)

0      -1
1      -1
2       0
3      -1
4       0
       ..
5446    1
5447    1
5448    1
5449    1
5450    1
Name: C, Length: 5451, dtype: category
Categories (3, int64): [-1, 0, 1]


In [ ]:
model=RandomForestClassifier()
params={'n_estimators':[300,400,500],'max_depth':[10,20,30],'min_samples_leaf':[2,3,4]}
gridcv=GridSearchCV(model, params, cv=5)
gridcv.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [2, 3, 4],
                         'n_estimators': [300, 400, 500]})

In [ ]:
print(gridcv.best_params_)

{'max_depth': 30, 'min_samples_leaf': 2, 'n_estimators': 500}


In [ ]:
model=RandomForestClassifier(n_estimators=500,max_depth=30,min_samples_leaf=2,class_weight={-1:0.2006369,1:0.126})
#class_weight={-1:0.2006369,0:0.02956359,1:0.126}
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

from sklearn.metrics import f1_score
score=f1_score(y_test, y_pred, average='macro')
print(score)

0.5803490781573775


###**test set과 최종 train set import**

In [ ]:
# total_trainset_smote version 가져오기
from google.colab import files
myfile=files.upload()

# total_smote_data.csv 불러오기

Saving total_smote_data.csv to total_smote_data.csv


In [ ]:
train_total = pd.read_csv(io.BytesIO(myfile['total_smote_data.csv']))

#data import하면 "Unnamed: 0"이라는 불필요한 열 생성 -> 제거 
train_total=train_total.drop(columns=['Unnamed: 0'])
train_total.head()

,Y_b,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,BX22,BX23,BX24,BX25,BX26,BX27,BX28,BX29,BX30,BX31,BX32,BX33,BX34,BX35,BX36,BX37,BX38,BX39,BX40,BX41,BX42,BX43,BX44,BX45,BX46,BX47,BX49,BX50,BX51,BX52,BX53,BX54,BX55,BX56,BX57,BX58,BX59,BX60,yy_b,C
0,19,22.383724,22.128057,20.546765,22.210202,8.331105,1.041042,-0.382176,58,0.174074,13.796952,-7.150123,-0.024824,0.010184,0.063153,0.039737,0.047798,0.018831,-0.621855,0.065031,0.064817,1.053593,-0.003308,-0.003841,0.114455,0.061723,-0.004960,0.885545,0.052732,-0.016081,17.013179,0.159301,0.010184,0.267607,0.234077,1.530844,0.713886,0.493392,0.932059,1.088378,...,-0.017948,-0.003323,0.115882,0.056666,-0.004603,0.884118,0.059216,-0.065824,14.162630,0.196878,0.016816,0.272419,0.241213,1.622491,0.753791,0.506855,0.993417,1.007513,3.869614,2.707154,1.372134,0.330942,0.632885,0.245141,-0.003509,0.024012,18.715408,0.780712,1.153126,0.010272,0.008250,-0.065530,-0.072887,15.046856,1.609134,0.722055,2.950170,7.179745,1997,-1
1,15,20.184873,19.906590,18.216023,20.034495,6.315358,0.880409,-0.545045,43,0.174074,13.591232,5.784040,0.010394,0.017520,0.057560,0.048016,0.042984,0.018831,0.189408,0.081739,0.079748,1.439674,-0.024963,0.001451,0.100284,0.056776,0.001917,0.899716,0.043508,-0.135363,17.013179,0.139617,0.017520,0.076551,0.136176,1.596068,0.945048,0.439255,1.094028,0.860245,...,-0.012946,0.002257,0.104049,0.062658,0.002779,0.895951,0.041390,-0.082338,12.543782,0.127709,0.019913,0.076551,0.258583,0.929468,0.933123,0.654066,1.258566,0.711496,6.123913,4.034517,2.309551,0.297859,0.708990,0.146406,0.136358,0.007096,18.223464,0.844509,0.563029,-0.030763,-0.026834,-0.034293,-0.036829,13.671127,1.444607,0.812241,2.966269,4.625441,1997,-1
2,12,21.026399,19.623518,17.221982,21.170458,7.671361,0.979021,-0.545045,42,0.174074,12.172528,-0.267213,0.240345,-0.129443,-0.041703,0.085964,-0.034289,0.003062,-5.264545,0.104237,0.097745,-0.945759,-0.116040,-0.101932,0.038197,-0.047666,-0.146520,0.961803,0.113158,0.274954,17.013179,0.118339,-0.129443,0.692466,0.011330,1.091126,0.319425,0.776332,0.495617,1.545983,...,-0.116040,-0.065861,0.087675,0.012465,-0.146520,0.912325,0.075210,-0.333624,12.822377,0.118339,-0.070151,0.692466,0.342592,0.689131,0.346003,1.229036,0.526314,1.361668,6.123913,4.034517,2.309551,0.344134,0.751389,0.133747,-0.205083,-0.023834,17.572121,0.058201,0.435633,-0.039685,-0.038299,-0.181887,-0.156031,37.349959,1.013982,0.370970,2.727980,5.430017,1997,0
3,16,21.322081,21.067553,20.126214,20.961911,8.131825,1.137434,-0.075908,42,0.174074,12.935729,-5.193739,0.004219,0.022513,0.074403,0.052825,0.057377,0.018831,0.138225,0.086533,0.085463,1.169247,-0.012526,0.001276,0.111693,0.074007,0.001646,0.888307,0.037686,-0.032108,14.673233,0.302442,0.022513,0.451714,0.198101,1.650138,0.598405,0.451540,0.746235,1.256871,...,-0.014650,0.006989,0.106502,0.069972,0.007435,0.893498,0.036530,-0.104062,6.313040,0.132332,0.030286,0.354025,0.241267,1.717064,0.653632,0.469319,0.861478,1.199768,5.899879,4.034517,2.094303,0.382395,0.660291,0.152515,-0.081800,0.029256,19.164520,0.830141,2.110466,0.101802,0.088331,0.070543,0.065115,9.999484,1.923207,0.939994,4.854726,5.030613,1997,-1
4,17,19.975171,19.635487,18.909031,19.553065,6.858565,0.932399,0.008580,42,0.239319,12.776922,-33.644871,-0.043416,0.004551,0.046257,0.049430,0.027739,0.011753,-3.537565,0.065937,0.064205,1.692427,-0.026977,-0.014950,0.188970,0.038960,-0.020997,0.811030,0.150010,-0.055782,7.956532,0.344335,0.004551,0.357530,0.335760,1.334028,0.875906,0.635180,1.318658,0.786697,...,-0.033657,-0.023878,0.166282,0.035335,-0.037372,0.833718,0.130947,-0.063286,4.588228,0.339781,0.027357,0.497970,0.381397,2.939148,0.557548,0.334706,0.901301,1.096611,1.762613,1.455807,0.487268,0.273604,0.411969,0.514648,-0.048822,0.004076,18.744605,0.600945,3.007393,0.134023,0.088484,0.069055,0.071358,15.552325,1.141874,0.638910,3.570284,8.855187,1997,0


In [ ]:
train_total['yy_b'].value_counts()

2001    474
1999    450
2000    438
2012    381
2002    375
2013    367
2006    361
2009    359
2014    343
2003    339
2007    332
2005    329
2010    323
2008    318
2011    317
2004    315
1998    268
2015    217
1997     87
Name: yy_b, dtype: int64

In [ ]:
# test_set 가져오기
from google.colab import files
myfile=files.upload()

# testset.csv 불러오기

Saving testset.csv to testset.csv


In [ ]:
test_set = pd.read_csv(io.BytesIO(myfile['testset.csv']))
test_set.head()
print(len(test_set))

778


In [ ]:
# train_set과 동일한 상황으로 만들어줌
id=test_set['Id']
test_set=test_set.drop(columns=['Id','yy_c','mm_c','mm_b'])
test_set.head()

,Y_b,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,BX21,BX22,BX23,BX24,BX25,BX26,BX27,BX28,BX29,BX30,BX31,BX32,BX33,BX34,BX35,BX36,BX37,BX38,BX39,BX40,BX41,BX42,BX43,BX44,BX45,BX46,BX47,BX49,BX50,BX51,BX52,BX53,BX54,BX55,BX56,BX57,BX58,BX59,BX60,yy_b
0,B-,18.818610,18.255515,17.954574,18.271367,5.170484,0.818093,-0.332493,65,2.422542,13.085031,22.084833,-0.159037,-0.129443,-0.041703,0.039835,-0.034289,0.001565,-0.056655,0.042582,0.037398,-0.443393,-0.116040,-0.067027,0.088813,-0.047666,-0.117707,0.911187,0.184851,-0.187294,0.566644,0.421457,-0.129443,0.395673,0.392281,3.076300,0.660292,0.212116,1.086509,0.921005,...,-0.217913,-0.116040,-0.101932,0.038197,-0.047666,-0.146520,0.961803,0.224210,-0.288641,0.403401,0.429973,-0.129443,0.421698,0.380952,3.077737,0.669202,0.193012,1.081018,0.925869,1.151921,1.111246,0.214484,0.387549,0.425630,0.754301,0.038711,-0.073576,18.138221,-1.153352,-2.147633,-0.099023,-0.065338,-0.181887,-0.156031,-4.325322,0.930674,0.511923,2.809160,14.732686,2016
1,AA-,21.876822,21.561147,21.338793,20.999998,4.976734,1.270982,-0.273974,65,0.563962,16.507898,3.326635,0.035226,0.283956,0.067584,0.007177,0.044491,0.002403,7.605750,0.010471,0.011029,0.171644,0.050535,0.020552,0.195354,0.061006,0.028180,0.804646,0.134348,0.063119,1.264650,0.583898,0.283956,0.364668,0.231019,0.764606,1.200129,0.790315,1.560675,0.830231,...,0.204638,0.039124,0.025401,0.187345,0.049190,0.035958,0.812655,0.138155,0.047254,0.922254,0.584864,0.288574,0.381949,0.249646,0.486356,1.346220,1.223447,1.794112,0.817695,0.627652,0.319234,0.390566,0.086315,0.270569,1.408847,0.148267,0.037928,19.034507,5.489111,5.115289,0.070489,0.029263,-0.003785,-0.002224,6.932102,1.926387,0.706399,9.353592,13.714503,2016
2,A,21.947360,21.359966,20.979382,21.469566,8.113427,0.403914,-0.124972,62,1.166677,13.246539,-2.328806,0.029891,0.086409,0.076038,0.021188,0.036461,0.000458,0.548200,0.021828,0.022827,0.332720,0.043776,0.011304,0.435844,0.065604,0.020340,0.564156,0.370241,0.064050,1.405546,0.379850,0.086409,0.618502,0.136059,1.454947,0.586846,0.496053,0.679266,1.227083,...,0.345141,0.045999,0.015420,0.430111,0.070243,0.027974,0.569889,0.359868,0.066241,1.285276,0.382779,0.097125,0.647731,0.162042,1.392740,0.497197,0.487564,0.593344,1.288388,1.492862,1.083969,0.528504,0.188719,0.381996,0.620165,-0.168730,0.050051,19.804265,3.378443,7.574758,0.142352,0.087862,0.101293,0.113422,8.460939,2.063609,0.551218,3.673679,6.984983,2016
3,AA-,21.439124,21.001501,21.126430,20.124317,7.414573,1.017514,-0.245187,65,1.580421,13.586929,1.136066,0.107832,0.523876,0.077179,-0.003208,0.047799,0.004676,14.458469,0.004035,0.004339,0.054503,0.070005,0.078814,0.294853,0.074041,0.122084,0.705147,0.220812,0.061784,0.280923,0.701803,0.523876,0.163224,0.296325,0.401783,1.951032,0.693378,2.465251,0.579492,...,0.036478,0.073274,0.067035,0.301816,0.076048,0.111671,0.698184,0.225768,0.060638,0.280923,0.701803,0.523876,0.163710,0.252609,0.762791,1.951032,0.447963,2.465251,0.604415,0.276728,0.261471,0.117129,0.034888,0.087804,2.346230,0.320557,0.083551,19.214152,28.257880,36.789995,0.244457,0.067134,0.089026,0.040729,1.741623,1.179199,0.600284,5.270520,5.203614,2016
4,AA-,22.431860,22.528590,21.730956,21.746410,8.596374,0.863289,-0.088236,65,1.490695,13.932217,8.654834,0.024901,0.098096,0.044244,0.007777,0.041376,0.000621,2.990054,0.008340,0.008592,0.222051,0.029221,0.012354,0.110346,0.037561,0.011215,0.889654,0.072785,0.064878,0.681276,0.496136,0.098096,0.368859,0.011330,0.402942,1.098002,1.041978,1.109195,0.964229,...,0.284892,0.026398,0.010884,0.108640,0.036915,0.009689,0.891360,0.071725,0.056273,0.488572,0.526988,0.107872,0.393867,0.011330,1.019540,0.956294,0.655894,0.963999,1.014396,0.855373,0.542049,0.355526,0.157817,0.423472,1.114113,-0.010284,0.038015,19.401805,4.070153,4.232513,0.080738,0.038190,0.048932,0.020603,8.806493,5.069560,1.123382,6.355444,11.534746,2016


In [ ]:
# Y_b 열을 문자열에서 숫자열로 대체
test_set['Y_b']=test_set['Y_b'].replace({'D':'0', 'C':'1','CC':'2','CCC-':'3','CCC':'4',
                                 'B-':'5','B':'6','B+':'7','BB-':'8','BB':'9',
                                 'BB+':'10','BBB-':'11','BBB':'12','BBB+':'13',
                                 'A-':'14','A':'15','A+':'16','AA-':'17','AA':'18',
                                 'AA+':'19','AAA':'20'})

test_set['Y_b'] = test_set['Y_b'].astype('int64')
test_set.head()

,Y_b,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,...,BX21,BX22,BX23,BX24,BX25,BX26,BX27,BX28,BX29,BX30,BX31,BX32,BX33,BX34,BX35,BX36,BX37,BX38,BX39,BX40,BX41,BX42,BX43,BX44,BX45,BX46,BX47,BX49,BX50,BX51,BX52,BX53,BX54,BX55,BX56,BX57,BX58,BX59,BX60,yy_b
0,5,18.818610,18.255515,17.954574,18.271367,5.170484,0.818093,-0.332493,65,2.422542,13.085031,22.084833,-0.159037,-0.129443,-0.041703,0.039835,-0.034289,0.001565,-0.056655,0.042582,0.037398,-0.443393,-0.116040,-0.067027,0.088813,-0.047666,-0.117707,0.911187,0.184851,-0.187294,0.566644,0.421457,-0.129443,0.395673,0.392281,3.076300,0.660292,0.212116,1.086509,0.921005,...,-0.217913,-0.116040,-0.101932,0.038197,-0.047666,-0.146520,0.961803,0.224210,-0.288641,0.403401,0.429973,-0.129443,0.421698,0.380952,3.077737,0.669202,0.193012,1.081018,0.925869,1.151921,1.111246,0.214484,0.387549,0.425630,0.754301,0.038711,-0.073576,18.138221,-1.153352,-2.147633,-0.099023,-0.065338,-0.181887,-0.156031,-4.325322,0.930674,0.511923,2.809160,14.732686,2016
1,17,21.876822,21.561147,21.338793,20.999998,4.976734,1.270982,-0.273974,65,0.563962,16.507898,3.326635,0.035226,0.283956,0.067584,0.007177,0.044491,0.002403,7.605750,0.010471,0.011029,0.171644,0.050535,0.020552,0.195354,0.061006,0.028180,0.804646,0.134348,0.063119,1.264650,0.583898,0.283956,0.364668,0.231019,0.764606,1.200129,0.790315,1.560675,0.830231,...,0.204638,0.039124,0.025401,0.187345,0.049190,0.035958,0.812655,0.138155,0.047254,0.922254,0.584864,0.288574,0.381949,0.249646,0.486356,1.346220,1.223447,1.794112,0.817695,0.627652,0.319234,0.390566,0.086315,0.270569,1.408847,0.148267,0.037928,19.034507,5.489111,5.115289,0.070489,0.029263,-0.003785,-0.002224,6.932102,1.926387,0.706399,9.353592,13.714503,2016
2,15,21.947360,21.359966,20.979382,21.469566,8.113427,0.403914,-0.124972,62,1.166677,13.246539,-2.328806,0.029891,0.086409,0.076038,0.021188,0.036461,0.000458,0.548200,0.021828,0.022827,0.332720,0.043776,0.011304,0.435844,0.065604,0.020340,0.564156,0.370241,0.064050,1.405546,0.379850,0.086409,0.618502,0.136059,1.454947,0.586846,0.496053,0.679266,1.227083,...,0.345141,0.045999,0.015420,0.430111,0.070243,0.027974,0.569889,0.359868,0.066241,1.285276,0.382779,0.097125,0.647731,0.162042,1.392740,0.497197,0.487564,0.593344,1.288388,1.492862,1.083969,0.528504,0.188719,0.381996,0.620165,-0.168730,0.050051,19.804265,3.378443,7.574758,0.142352,0.087862,0.101293,0.113422,8.460939,2.063609,0.551218,3.673679,6.984983,2016
3,17,21.439124,21.001501,21.126430,20.124317,7.414573,1.017514,-0.245187,65,1.580421,13.586929,1.136066,0.107832,0.523876,0.077179,-0.003208,0.047799,0.004676,14.458469,0.004035,0.004339,0.054503,0.070005,0.078814,0.294853,0.074041,0.122084,0.705147,0.220812,0.061784,0.280923,0.701803,0.523876,0.163224,0.296325,0.401783,1.951032,0.693378,2.465251,0.579492,...,0.036478,0.073274,0.067035,0.301816,0.076048,0.111671,0.698184,0.225768,0.060638,0.280923,0.701803,0.523876,0.163710,0.252609,0.762791,1.951032,0.447963,2.465251,0.604415,0.276728,0.261471,0.117129,0.034888,0.087804,2.346230,0.320557,0.083551,19.214152,28.257880,36.789995,0.244457,0.067134,0.089026,0.040729,1.741623,1.179199,0.600284,5.270520,5.203614,2016
4,17,22.431860,22.528590,21.730956,21.746410,8.596374,0.863289,-0.088236,65,1.490695,13.932217,8.654834,0.024901,0.098096,0.044244,0.007777,0.041376,0.000621,2.990054,0.008340,0.008592,0.222051,0.029221,0.012354,0.110346,0.037561,0.011215,0.889654,0.072785,0.064878,0.681276,0.496136,0.098096,0.368859,0.011330,0.402942,1.098002,1.041978,1.109195,0.964229,...,0.284892,0.026398,0.010884,0.108640,0.036915,0.009689,0.891360,0.071725,0.056273,0.488572,0.526988,0.107872,0.393867,0.011330,1.019540,0.956294,0.655894,0.963999,1.014396,0.855373,0.542049,0.355526,0.157817,0.423472,1.114113,-0.010284,0.038015,19.401805,4.070153,4.232513,0.080738,0.038190,0.048932,0.020603,8.806493,5.069560,1.123382,6.355444,11.534746,2016


###**최종 train set, test set 지정**

In [ ]:
x_train=train_total.drop(columns=['C']).astype(float)
y_train = train_total['C'].astype('category')
  
x_test=test_set.astype(float)

In [ ]:
print(y_train)

0      -1
1      -1
2       0
3      -1
4       0
       ..
6388    1
6389    1
6390    1
6391    1
6392    1
Name: C, Length: 6393, dtype: category
Categories (3, int64): [-1, 0, 1]


###**RandomForest model - test set predictions**

In [ ]:
model=RandomForestClassifier(n_estimators=500,max_depth=30,min_samples_leaf=2,class_weight={-1:0.2006369,1:0.126})
model.fit(x_train,y_train)
y_pred=model.predict(x_test)

In [ ]:
#print(predictions)
#print(len(y_pred))
print(y_pred)
#predictions=np.argmax(y_pred,axis=1)-1

[-1  0  0  0  0  0 -1  0  0  0  0  0  0 -1  0  0 -1  0  0  0  0  1  0  0
  0 -1  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0
  0  0  1  0  0 -1  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 -1
  0  0  0  0  0 -1  0  0  0  0  0 -1  0  1  0  0  0 -1  0  0  0 -1  0  0
  0  0  1  0  0 -1  0  0  0  0 -1  0  1  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0 -1  0  0  0  0  0  0  0  0  0 -1  0 -1  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0 -1  0  0 -1  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1  0  0  0 -1  0
  0  0  0  0  0  0 -1  0  0 -1  0  0 -1  0  0  0  0  0  0  0  0  0  0  0
  0  0 -1  0  0  0  0  0 -1 -1  0  1  0  0  0 -1 -1  0  0 -1  0  0  0  0
  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0 -1  0  0  0  0  0  0 -1  0  1  0  0  0 -1  0  0  0  0  0  0  0 -1 -1
  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1 -1  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0

In [ ]:
submit=pd.DataFrame({'Id':id,'Predicted':y_pred})
submit.head()

,Id,Predicted
0,b1,-1
1,b2,0
2,b3,0
3,b4,0
4,b5,0


In [ ]:
submit.to_csv("rf_pred.csv",mode='w')